In [2]:
import torch

In [3]:
name = 'tinycodes'
weights = torch.load(f'../weight_analysis_results/{name}.pt')

In [4]:
checkpoints = ["pretrained"]
step_interval = 500
max_step = 8500
for i in range((max_step // step_interval) + 1):
    checkpoints.append(f"/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-{name}-gpt2-0.0008-1-{name}_560000_step-{i * step_interval}_finetuned.pt")

print(checkpoints)

['pretrained', '/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-tinycodes-gpt2-0.0008-1-tinycodes_560000_step-0_finetuned.pt', '/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-tinycodes-gpt2-0.0008-1-tinycodes_560000_step-500_finetuned.pt', '/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-tinycodes-gpt2-0.0008-1-tinycodes_560000_step-1000_finetuned.pt', '/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-tinycodes-gpt2-0.0008-1-tinycodes_560000_step-1500_finetuned.pt', '/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-tinycodes-gpt2-0.0008-1-tinycodes_560000_step-2000_finetuned.pt', '/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-tinycodes-gpt2-0.0008-1-tinycodes_560000_step-2500_finetuned.pt', '/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-tinycodes-gpt2-0.0008-1-tinycodes_560000_step-3000_finetuned.pt', '/data/scratch-oc40/pulkitag/jyop/gpt-merge-out/gpt2-tinycodes-gpt2-0.0008-1-tinycodes_560000_step-3500_finetuned.pt', '/data/scratch-oc40/pulkitag/jyop/gpt

In [5]:
def calculate_stable_rank(matrix):
    frobenius_norm = torch.linalg.norm(matrix, 'fro')
    spectral_norm = torch.linalg.norm(matrix, 2)
    return (frobenius_norm ** 2) / (spectral_norm ** 2)

In [44]:
layer = 11
# prev_in_proj = 0
# for chkpt in checkpoints:
#     in_proj = weights[chkpt][layer]['in_proj']
#     out_proj = weights[chkpt][layer]['out_proj']

#     delta = in_proj - prev_in_proj
#     print(f"delta: {calculate_stable_rank(delta)}")
#     prev_in_proj = in_proj

chkpt0 = checkpoints[0]
chkpt1 = checkpoints[5]
chkpt2 = checkpoints[17]


in_proj_0 = weights[chkpt0][layer]['in_proj']
in_proj_1 = weights[chkpt1][layer]['in_proj']
in_proj_2 = weights[chkpt2][layer]['in_proj']

out_proj_0 = weights[chkpt0][layer]['out_proj']
out_proj_1 = weights[chkpt1][layer]['out_proj']
out_proj_2 = weights[chkpt2][layer]['out_proj']


delta_in_proj_1 = in_proj_1 - in_proj_0
delta_out_proj_1 = out_proj_1 - out_proj_0

delta_in_proj_2 = in_proj_2 - in_proj_1
delta_out_proj_2 = out_proj_2 - out_proj_1
# print(f"delta: {calculate_stable_rank(delta_in_proj_1)}")
# print(f"delta: {calculate_stable_rank(delta_out_proj_1)}")
# print(f"delta: {calculate_stable_rank(delta_in_proj_2)}")
# print(f"delta: {calculate_stable_rank(delta_out_proj_2)}")


# compute svd of the deltas
u_in_1, s_in_1, v_in_1 = torch.linalg.svd(delta_in_proj_1)
u_out_1, s_out_1, v_out_1 = torch.linalg.svd(delta_out_proj_1)

u_in_2, s_in_2, v_in_2 = torch.linalg.svd(delta_in_proj_2)
u_out_2, s_out_2, v_out_2 = torch.linalg.svd(delta_out_proj_2)


In [45]:
# compute distance between the singular vectors
delta_u_in = u_in_2 - u_in_1
delta_s_in = s_in_2 - s_in_1
delta_v_in = v_in_2 - v_in_1

delta_u_out = u_out_2 - u_out_1
delta_s_out = s_out_2 - s_out_1
delta_v_out = v_out_2 - v_out_1


print(torch.linalg.norm(delta_u_in, 'fro'))
print(torch.linalg.norm(delta_v_in, 'fro'))
print(torch.linalg.norm(delta_u_out, 'fro'))
print(torch.linalg.norm(delta_v_out, 'fro'))


tensor(57.2620)
tensor(39.1520)
tensor(39.1906)
tensor(56.3204)


In [46]:
import torch.optim as optim
results = {}
for layer in range(12):
    # prev_in_proj = 0
    # for chkpt in checkpoints:
    #     in_proj = weights[chkpt][layer]['in_proj']
    #     out_proj = weights[chkpt][layer]['out_proj']

    #     delta = in_proj - prev_in_proj
    #     print(f"delta: {calculate_stable_rank(delta)}")
    #     prev_in_proj = in_proj

    chkpt0 = checkpoints[0]
    chkpt1 = checkpoints[5]
    chkpt2 = checkpoints[17]


    in_proj_0 = weights[chkpt0][layer]['in_proj']
    in_proj_1 = weights[chkpt1][layer]['in_proj']
    in_proj_2 = weights[chkpt2][layer]['in_proj']

    out_proj_0 = weights[chkpt0][layer]['out_proj']
    out_proj_1 = weights[chkpt1][layer]['out_proj']
    out_proj_2 = weights[chkpt2][layer]['out_proj']


    delta_in_proj_1 = in_proj_1 - in_proj_0
    delta_out_proj_1 = out_proj_1 - out_proj_0

    delta_in_proj_2 = in_proj_2 - in_proj_1
    delta_out_proj_2 = out_proj_2 - out_proj_1
    # print(f"delta: {calculate_stable_rank(delta_in_proj_1)}")
    # print(f"delta: {calculate_stable_rank(delta_out_proj_1)}")
    # print(f"delta: {calculate_stable_rank(delta_in_proj_2)}")
    # print(f"delta: {calculate_stable_rank(delta_out_proj_2)}")


    # compute svd of the deltas
    u_in_1, s_in_1, v_in_1 = torch.linalg.svd(delta_in_proj_1)
    u_out_1, s_out_1, v_out_1 = torch.linalg.svd(delta_out_proj_1)

    u_in_2, s_in_2, v_in_2 = torch.linalg.svd(delta_in_proj_2)
    u_out_2, s_out_2, v_out_2 = torch.linalg.svd(delta_out_proj_2)

    # compute distance between the singular vectors
    delta_u_in = u_in_2 - u_in_1
    delta_s_in = s_in_2 - s_in_1
    delta_v_in = v_in_2 - v_in_1

    delta_u_out = u_out_2 - u_out_1
    delta_s_out = s_out_2 - s_out_1
    delta_v_out = v_out_2 - v_out_1

    A = delta_out_proj_2
    B = delta_out_proj_1

    # Compute the SVD of A
    U_A, S_A, V_A = torch.svd(A)

    # Make S_A a learnable parameter
    S_A = torch.nn.Parameter(S_A)

    # Optimizer to update only the singular values of A
    optimizer = optim.Adam([S_A], lr=0.05)

    # Number of iterations for optimization
    num_iterations = 100

    losses = []
    for i in range(num_iterations):
        optimizer.zero_grad()
        
        # Reconstruct A using the fixed U_A, V_A, and the learnable S_A
        A_reconstructed = torch.mm(U_A, torch.mm(torch.diag(S_A), V_A.t()))
        
        # Compute loss as the Frobenius norm of the difference between A_reconstructed and B
        loss = torch.norm(A_reconstructed - B, p='fro')
        print(f"Iteration {i}: Loss = {loss.item()}")
        losses.append(loss.item())
        
        # Backpropagate the loss and update S_A
        loss.backward()
        optimizer.step()
    
    # Store the results
    results[layers] = {losses}
    #if i % 100 == 0:


Iteration 0: Loss = 62.153987884521484
Iteration 1: Loss = 61.729827880859375
Iteration 2: Loss = 61.334678649902344
Iteration 3: Loss = 60.9699821472168
Iteration 4: Loss = 60.635963439941406
Iteration 5: Loss = 60.333133697509766
Iteration 6: Loss = 60.061431884765625
Iteration 7: Loss = 59.82054138183594
Iteration 8: Loss = 59.609832763671875
Iteration 9: Loss = 59.427330017089844
Iteration 10: Loss = 59.27156066894531
Iteration 11: Loss = 59.13985061645508
Iteration 12: Loss = 59.029998779296875
Iteration 13: Loss = 58.93935012817383
Iteration 14: Loss = 58.86532211303711
Iteration 15: Loss = 58.805564880371094
Iteration 16: Loss = 58.757469177246094
Iteration 17: Loss = 58.71942138671875
Iteration 18: Loss = 58.68952941894531
Iteration 19: Loss = 58.666160583496094
Iteration 20: Loss = 58.648048400878906
Iteration 21: Loss = 58.633941650390625
Iteration 22: Loss = 58.62344741821289
Iteration 23: Loss = 58.61518478393555
Iteration 24: Loss = 58.60897445678711
Iteration 25: Loss = 5